## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.65,115.33,59.18,70,86,20.40,overcast clouds
1,1,Cape Town,ZA,-33.93,18.42,57.88,81,40,5.75,light rain
2,2,Vaini,TO,-21.20,-175.20,75.36,100,75,9.22,broken clouds
3,3,Lamu,KE,-2.27,40.90,77.22,76,45,16.75,scattered clouds
4,4,Maumere,ID,-8.62,122.21,78.51,80,100,6.78,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Vaini,TO,-21.20,-175.20,75.36,100,75,9.22,broken clouds
3,3,Lamu,KE,-2.27,40.90,77.22,76,45,16.75,scattered clouds
4,4,Maumere,ID,-8.62,122.21,78.51,80,100,6.78,overcast clouds
8,8,Ardakan,IR,32.31,54.02,88.77,14,0,2.08,clear sky
10,10,Port Hawkesbury,CA,45.62,-61.35,73.90,80,0,11.50,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                296
City                   296
Country                296
Lat                    296
Lng                    296
Max Temp               296
Humidity               296
Cloudiness             296
Wind Speed             296
Current Description    296
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vaini,TO,75.36,broken clouds,-21.20,-175.20,
3,Lamu,KE,77.22,scattered clouds,-2.27,40.90,
4,Maumere,ID,78.51,overcast clouds,-8.62,122.21,
8,Ardakan,IR,88.77,clear sky,32.31,54.02,
10,Port Hawkesbury,CA,73.90,clear sky,45.62,-61.35,
14,Bengkulu,ID,75.51,broken clouds,-3.80,102.27,
15,Namatanai,PG,77.88,clear sky,-3.67,152.43,
19,Puerto Escondido,MX,86.45,overcast clouds,15.85,-97.07,
20,Amapa,BR,80.62,moderate rain,1.00,-52.00,
22,Kabo,CF,73.11,overcast clouds,7.70,18.63,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   296
Country                296
Max Temp               296
Current Description    296
Lat                    296
Lng                    296
Hotel Name             296
dtype: int64

In [13]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vaini,TO,75.36,broken clouds,-21.20,-175.20,Keleti Beach Resort
3,Lamu,KE,77.22,scattered clouds,-2.27,40.90,Lamu House
4,Maumere,ID,78.51,overcast clouds,-8.62,122.21,Sylvia Hotel
8,Ardakan,IR,88.77,clear sky,32.31,54.02,Ardakan Hotel
10,Port Hawkesbury,CA,73.90,clear sky,45.62,-61.35,MacKenzie House Tourist Home B&B


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt<dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))